In [1]:
import sys
sys.path.append("../")
from utils import train_test_split

from tqdm.auto import tqdm 
tqdm.pandas()
import datetime
import pandas as pd
from prophet import Prophet

DATA = "../data/"

data = pd.read_csv(DATA + "fact_train_test.csv", sep=";").sort_values(by=["period"])
data

,period,rps,podrod,filial,client_sap_id,freight_id,sender_station_id,recipient_station_id,sender_organisation_id,real_weight,real_wagon_count
0,2012-07-01,1,5,1,328,1193,30252,13005,10036,"71,0",1
1,2012-10-01,1,5,1,328,1193,30252,11376,10036,"210,0",3
2,2014-03-01,0,1,1,328,3472,30252,29548,10036,"67,0",1
3,2014-03-01,0,1,1,328,3472,30252,29158,10036,"67,0",1
4,2014-03-01,0,2,1,328,3472,30252,27484,10036,"66,0",1
...,...,...,...,...,...,...,...,...,...,...,...
3527287,2023-03-01,0,2,2,2129,1697,35809,30792,25817,"194,0",3
3527288,2023-03-01,0,2,2,2129,1697,35809,31112,25817,"65,0",1
3527289,2023-03-01,0,2,2,2129,1708,35667,38470,25817,"659,0",10
3527280,2023-03-01,0,2,2,2129,2120,35809,36430,25817,"66,0",1


In [2]:
grouped_data = data.groupby(
    by=["rps", "podrod", "filial", "client_sap_id", "freight_id", "sender_station_id", "recipient_station_id", "sender_organisation_id"]
).agg({"real_wagon_count": list, "period": list}).reset_index()

grouped_data

,rps,podrod,filial,client_sap_id,freight_id,sender_station_id,recipient_station_id,sender_organisation_id,real_wagon_count,period
0,0,0,0,-1,12,37642,38641,20215,[0],[2020-07-01]
1,0,0,0,-1,12,38641,37642,20215,"[0, 0]","[2020-02-01, 2020-03-01]"
2,0,0,0,-1,15,38963,38966,30964,"[6, 4, 7, 3, 6, 6]","[2022-07-01, 2022-08-01, 2022-09-01, 2022-10-0..."
3,0,0,0,-1,18,38830,33818,8653,[1],[2021-12-01]
4,0,0,0,-1,18,38830,33818,20153,"[1, 0, 1, 0, 0]","[2020-02-01, 2020-07-01, 2020-08-01, 2021-01-0..."
...,...,...,...,...,...,...,...,...,...,...
1025493,1,5,2,2415,2997,13005,37870,10441,"[1, 2]","[2018-05-01, 2018-06-01]"
1025494,1,5,2,2415,3471,32800,12142,7178,[1],[2019-03-01]
1025495,1,5,2,2415,3471,32800,12840,7178,[1],[2018-11-01]
1025496,1,5,2,2416,1691,27096,27484,10579,"[2, 2]","[2021-04-01, 2021-05-01]"


In [3]:
needed_samples = pd.read_csv(DATA + "forecast_example.csv", sep=";")[["rps", "podrod", "filial", "client_sap_id", "freight_id", "sender_station_id", "recipient_station_id", "sender_organisation_id", "period"]]
needed_samples = needed_samples.groupby(
    by=["rps", "podrod", "filial", "client_sap_id", "freight_id", "sender_station_id", "recipient_station_id", "sender_organisation_id"]
).agg({"period": list}).reset_index()
needed_samples

,rps,podrod,filial,client_sap_id,freight_id,sender_station_id,recipient_station_id,sender_organisation_id,period
0,0,0,0,-1,349,38725,38966,27437,"[2023-04-01, 2023-05-01, 2023-06-01, 2023-07-0..."
1,0,0,0,-1,349,38754,38966,26664,"[2023-04-01, 2023-05-01, 2023-06-01, 2023-07-0..."
2,0,0,0,-1,349,38756,38966,26664,"[2023-04-01, 2023-05-01, 2023-06-01, 2023-07-0..."
3,0,0,0,-1,349,38756,38966,27437,"[2023-04-01, 2023-05-01, 2023-06-01, 2023-07-0..."
4,0,0,0,-1,349,38760,38966,26664,"[2023-04-01, 2023-05-01, 2023-06-01, 2023-07-0..."
...,...,...,...,...,...,...,...,...,...
47916,1,5,2,2408,1691,11624,23540,25846,"[2023-04-01, 2023-05-01, 2023-06-01, 2023-07-0..."
47917,1,5,2,2408,1691,11624,27374,25846,"[2023-04-01, 2023-05-01, 2023-06-01, 2023-07-0..."
47918,1,5,2,2408,1691,11624,32502,25846,"[2023-04-01, 2023-05-01, 2023-06-01, 2023-07-0..."
47919,1,5,2,2408,1691,11624,35151,25846,"[2023-04-01, 2023-05-01, 2023-06-01, 2023-07-0..."


In [4]:
needed_samples_ty = needed_samples.merge(
    grouped_data, 
    on=["rps", "podrod", "filial", "client_sap_id", "freight_id", "sender_station_id", "recipient_station_id", "sender_organisation_id"],
    how="left"
)

needed_samples_ty

,rps,podrod,filial,client_sap_id,freight_id,sender_station_id,recipient_station_id,sender_organisation_id,period_x,real_wagon_count,period_y
0,0,0,0,-1,349,38725,38966,27437,"[2023-04-01, 2023-05-01, 2023-06-01, 2023-07-0...","[19, 10, 11, 7, 4, 3, 8, 7]","[2022-08-01, 2022-09-01, 2022-10-01, 2022-11-0..."
1,0,0,0,-1,349,38754,38966,26664,"[2023-04-01, 2023-05-01, 2023-06-01, 2023-07-0...",[1],[2023-03-01]
2,0,0,0,-1,349,38756,38966,26664,"[2023-04-01, 2023-05-01, 2023-06-01, 2023-07-0...","[5, 1, 1, 3, 4, 2, 1, 2]","[2022-03-01, 2022-08-01, 2022-09-01, 2022-10-0..."
3,0,0,0,-1,349,38756,38966,27437,"[2023-04-01, 2023-05-01, 2023-06-01, 2023-07-0...","[1, 2]","[2023-02-01, 2023-03-01]"
4,0,0,0,-1,349,38760,38966,26664,"[2023-04-01, 2023-05-01, 2023-06-01, 2023-07-0...","[1, 4, 2, 11, 9, 5, 7, 10]","[2022-08-01, 2022-09-01, 2022-10-01, 2022-11-0..."
...,...,...,...,...,...,...,...,...,...,...,...
47916,1,5,2,2408,1691,11624,23540,25846,"[2023-04-01, 2023-05-01, 2023-06-01, 2023-07-0...","[2, 2, 1, 1, 2, 1, 1]","[2022-01-01, 2022-02-01, 2022-03-01, 2022-07-0..."
47917,1,5,2,2408,1691,11624,27374,25846,"[2023-04-01, 2023-05-01, 2023-06-01, 2023-07-0...","[5, 6, 9, 1, 3, 3, 5, 4, 1, 2]","[2022-01-01, 2022-02-01, 2022-03-01, 2022-04-0..."
47918,1,5,2,2408,1691,11624,32502,25846,"[2023-04-01, 2023-05-01, 2023-06-01, 2023-07-0...","[4, 1, 1]","[2022-04-01, 2023-02-01, 2023-03-01]"
47919,1,5,2,2408,1691,11624,35151,25846,"[2023-04-01, 2023-05-01, 2023-06-01, 2023-07-0...","[1, 1, 1]","[2022-12-01, 2023-02-01, 2023-03-01]"


In [5]:
ty = needed_samples_ty[["period_y", "real_wagon_count"]].rename(columns={"period_y": "ds", "real_wagon_count": "y"})
ty

,ds,y
0,"[2022-08-01, 2022-09-01, 2022-10-01, 2022-11-0...","[19, 10, 11, 7, 4, 3, 8, 7]"
1,[2023-03-01],[1]
2,"[2022-03-01, 2022-08-01, 2022-09-01, 2022-10-0...","[5, 1, 1, 3, 4, 2, 1, 2]"
3,"[2023-02-01, 2023-03-01]","[1, 2]"
4,"[2022-08-01, 2022-09-01, 2022-10-01, 2022-11-0...","[1, 4, 2, 11, 9, 5, 7, 10]"
...,...,...
47916,"[2022-01-01, 2022-02-01, 2022-03-01, 2022-07-0...","[2, 2, 1, 1, 2, 1, 1]"
47917,"[2022-01-01, 2022-02-01, 2022-03-01, 2022-04-0...","[5, 6, 9, 1, 3, 3, 5, 4, 1, 2]"
47918,"[2022-04-01, 2023-02-01, 2023-03-01]","[4, 1, 1]"
47919,"[2022-12-01, 2023-02-01, 2023-03-01]","[1, 1, 1]"


In [6]:
dfs = [pd.DataFrame(dict(ty.iloc[i])) for i in tqdm(range(ty.shape[0]))]
dfs

  0%|          | 0/47921 [00:00<?, ?it/s]

[           ds   y
 0  2022-08-01  19
 1  2022-09-01  10
 2  2022-10-01  11
 3  2022-11-01   7
 4  2022-12-01   4
 5  2023-01-01   3
 6  2023-02-01   8
 7  2023-03-01   7,
            ds  y
 0  2023-03-01  1,
            ds  y
 0  2022-03-01  5
 1  2022-08-01  1
 2  2022-09-01  1
 3  2022-10-01  3
 4  2022-11-01  4
 5  2022-12-01  2
 6  2023-01-01  1
 7  2023-03-01  2,
            ds  y
 0  2023-02-01  1
 1  2023-03-01  2,
            ds   y
 0  2022-08-01   1
 1  2022-09-01   4
 2  2022-10-01   2
 3  2022-11-01  11
 4  2022-12-01   9
 5  2023-01-01   5
 6  2023-02-01   7
 7  2023-03-01  10,
            ds  y
 0  2023-02-01  3
 1  2023-03-01  1,
             ds  y
 0   2022-01-01  0
 1   2022-02-01  0
 2   2022-03-01  0
 3   2022-04-01  0
 4   2022-05-01  0
 5   2022-06-01  1
 6   2022-07-01  0
 7   2022-08-01  0
 8   2022-09-01  0
 9   2022-10-01  0
 10  2023-01-01  0
 11  2023-02-01  0
 12  2023-03-01  0,
             ds  y
 0   2022-01-01  0
 1   2022-02-01  0
 2   2022-03-01  0
 3 

In [7]:
lens = [el.shape[0] for el in dfs]
lens

[8,
 1,
 8,
 2,
 8,
 2,
 13,
 11,
 11,
 2,
 1,
 12,
 13,
 13,
 13,
 3,
 7,
 4,
 7,
 4,
 9,
 11,
 8,
 7,
 3,
 10,
 8,
 13,
 15,
 14,
 8,
 4,
 1,
 6,
 3,
 1,
 11,
 6,
 13,
 13,
 11,
 11,
 11,
 11,
 4,
 3,
 1,
 2,
 4,
 4,
 15,
 10,
 5,
 2,
 2,
 2,
 1,
 1,
 1,
 2,
 5,
 7,
 2,
 14,
 10,
 1,
 3,
 5,
 1,
 7,
 6,
 15,
 2,
 1,
 6,
 15,
 2,
 1,
 1,
 1,
 1,
 11,
 9,
 11,
 1,
 1,
 1,
 1,
 5,
 15,
 5,
 13,
 8,
 1,
 9,
 7,
 4,
 6,
 1,
 1,
 3,
 1,
 4,
 11,
 11,
 9,
 1,
 12,
 4,
 2,
 14,
 2,
 7,
 1,
 1,
 2,
 2,
 5,
 10,
 15,
 15,
 7,
 12,
 12,
 15,
 15,
 11,
 11,
 10,
 2,
 7,
 12,
 6,
 8,
 1,
 8,
 3,
 9,
 7,
 11,
 12,
 12,
 8,
 15,
 15,
 8,
 15,
 12,
 9,
 6,
 4,
 10,
 6,
 10,
 9,
 6,
 15,
 15,
 8,
 15,
 4,
 13,
 13,
 6,
 15,
 2,
 12,
 10,
 15,
 14,
 15,
 14,
 5,
 7,
 8,
 9,
 14,
 12,
 9,
 12,
 13,
 7,
 15,
 13,
 7,
 6,
 8,
 2,
 1,
 1,
 12,
 7,
 7,
 14,
 6,
 4,
 13,
 11,
 1,
 15,
 8,
 11,
 16,
 14,
 16,
 2,
 8,
 13,
 11,
 11,
 11,
 2,
 7,
 5,
 9,
 3,
 4,
 9,
 14,
 9,
 2,
 5,
 1,
 6,
 11,
 2,
 11,
 1,
 

In [8]:
needed_dates = needed_samples_ty["period_x"]
periods = [pd.DataFrame({"ds": needed_dates[i]}) for i in tqdm(range(len(needed_dates)))]
periods

  0%|          | 0/47921 [00:00<?, ?it/s]

[           ds
 0  2023-04-01
 1  2023-05-01
 2  2023-06-01
 3  2023-07-01
 4  2023-08-01,
            ds
 0  2023-04-01
 1  2023-05-01
 2  2023-06-01
 3  2023-07-01
 4  2023-08-01,
            ds
 0  2023-04-01
 1  2023-05-01
 2  2023-06-01
 3  2023-07-01
 4  2023-08-01,
            ds
 0  2023-04-01
 1  2023-05-01
 2  2023-06-01
 3  2023-07-01
 4  2023-08-01,
            ds
 0  2023-04-01
 1  2023-05-01
 2  2023-06-01
 3  2023-07-01
 4  2023-08-01,
            ds
 0  2023-04-01
 1  2023-05-01
 2  2023-06-01
 3  2023-07-01
 4  2023-08-01,
            ds
 0  2023-04-01
 1  2023-05-01
 2  2023-06-01
 3  2023-07-01
 4  2023-08-01,
            ds
 0  2023-04-01
 1  2023-05-01
 2  2023-06-01
 3  2023-07-01
 4  2023-08-01,
            ds
 0  2023-04-01
 1  2023-05-01
 2  2023-06-01
 3  2023-07-01
 4  2023-08-01,
            ds
 0  2023-04-01
 1  2023-05-01
 2  2023-06-01
 3  2023-07-01
 4  2023-08-01,
            ds
 0  2023-04-01
 1  2023-05-01
 2  2023-06-01
 3  2023-07-01
 4  2023-08-01,

In [9]:
len(periods), len(dfs)

(47921, 47921)

In [10]:
from multiprocessing import Pool

In [11]:
concat = list(zip(dfs, periods, lens))

In [12]:
def giga_apply(df, period, lenz):
    if lenz >= 2: 
        forecast = pd.DataFrame({"yhat": [df["y"][0]] * 5})
        model = Prophet()
        model.fit(df)
        return model.predict(period)["yhat"]
        
    else:
        return pd.DataFrame({"yhat": [df["y"][0]] * 5})


In [ ]:
results = []
with Pool(12) as pool:
    pool_results = pool.starmap_async(giga_apply, tqdm(concat))
    for result in pool_results.get():
        results.append(result)

  0%|          | 0/47921 [00:00<?, ?it/s]

04:48:30 - cmdstanpy - INFO - Chain [1] start processing
04:48:30 - cmdstanpy - INFO - Chain [1] done processing
04:48:30 - cmdstanpy - INFO - Chain [1] done processing
04:48:30 - cmdstanpy - INFO - Chain [1] start processing
04:48:30 - cmdstanpy - INFO - Chain [1] start processing
04:48:30 - cmdstanpy - INFO - Chain [1] done processing
04:48:30 - cmdstanpy - INFO - Chain [1] done processing
04:48:30 - cmdstanpy - INFO - Chain [1] start processing
04:48:30 - cmdstanpy - INFO - Chain [1] start processing
04:48:30 - cmdstanpy - INFO - Chain [1] start processing
04:48:30 - cmdstanpy - INFO - Chain [1] done processing
04:48:30 - cmdstanpy - INFO - Chain [1] start processing
04:48:30 - cmdstanpy - INFO - Chain [1] done processing
04:48:30 - cmdstanpy - INFO - Chain [1] start processing
04:48:30 - cmdstanpy - INFO - Chain [1] start processing
04:48:30 - cmdstanpy - INFO - Chain [1] done processing
04:48:30 - cmdstanpy - INFO - Chain [1] done processing
04:48:30 - cmdstanpy - INFO - Chain [1]

In [29]:
results[10]

0    1
1    1
2    1
3    1
4    1
Name: yhat, dtype: int64

In [30]:
fixed_results = []

for result in tqdm(results):
    if type(result) == pd.DataFrame:
        result = result["yhat"]

    result[result < 0] = 0
    fixed_results.append(result)

  0%|          | 0/47921 [00:00<?, ?it/s]

In [32]:
fixed_results[0]

0    2.170165
1    0.748055
2    0.000000
3    0.000000
4    0.000000
Name: yhat, dtype: float64

In [33]:
fixed_results_2 = [list(res) for res in fixed_results]
fixed_results_2

[[2.1701646838428066, 0.7480550185620839, 0.0, 0.0, 0.0],
 [1, 1, 1, 1, 1],
 [1.1585308030267298,
  0.9538831408432147,
  0.7424138899202498,
  0.5377662277367351,
  0.3262969768137697],
 [3.1071444164285715,
  4.178573780714285,
  5.285717457142857,
  6.357146821428571,
  7.464290497857142],
 [10.648649103590703,
  11.64578146419374,
  12.676151570150212,
  13.67328393075325,
  14.703654036709722],
 [0.0, 0.0, 0.0, 0.0, 0.0],
 [0.02695802250398005,
  0.021152463552824186,
  0.015153385969963101,
  0.009347827018807237,
  0.003348749435946166],
 [0.0, 0.0, 0.0, 0.0, 0.0],
 [0.0, 0.0, 0.0, 0.0, 0.0],
 [0.0, 0.0, 0.0, 0.0, 0.0],
 [1, 1, 1, 1, 1],
 [3.611306629582276,
  3.7759501412808745,
  3.946081770036093,
  4.110725281734691,
  4.280856910489909],
 [2.2201937854314293,
  2.210287771232286,
  2.2000515565598375,
  2.1901455423606944,
  2.179909327688246],
 [2.2681797398684265,
  2.1660343376214826,
  2.0604840886329736,
  1.9583386863860297,
  1.852788437397521],
 [3.226673150002593,


In [35]:
needed_samples_ty["forecast_wagon_count"] = fixed_results_2
needed_samples_ty

,rps,podrod,filial,client_sap_id,freight_id,sender_station_id,recipient_station_id,sender_organisation_id,period_x,real_wagon_count,period_y,forecast_wagon_count
0,0,0,0,-1,349,38725,38966,27437,"[2023-04-01, 2023-05-01, 2023-06-01, 2023-07-0...","[19, 10, 11, 7, 4, 3, 8, 7]","[2022-08-01, 2022-09-01, 2022-10-01, 2022-11-0...","[2.1701646838428066, 0.7480550185620839, 0.0, ..."
1,0,0,0,-1,349,38754,38966,26664,"[2023-04-01, 2023-05-01, 2023-06-01, 2023-07-0...",[1],[2023-03-01],"[1, 1, 1, 1, 1]"
2,0,0,0,-1,349,38756,38966,26664,"[2023-04-01, 2023-05-01, 2023-06-01, 2023-07-0...","[5, 1, 1, 3, 4, 2, 1, 2]","[2022-03-01, 2022-08-01, 2022-09-01, 2022-10-0...","[1.1585308030267298, 0.9538831408432147, 0.742..."
3,0,0,0,-1,349,38756,38966,27437,"[2023-04-01, 2023-05-01, 2023-06-01, 2023-07-0...","[1, 2]","[2023-02-01, 2023-03-01]","[3.1071444164285715, 4.178573780714285, 5.2857..."
4,0,0,0,-1,349,38760,38966,26664,"[2023-04-01, 2023-05-01, 2023-06-01, 2023-07-0...","[1, 4, 2, 11, 9, 5, 7, 10]","[2022-08-01, 2022-09-01, 2022-10-01, 2022-11-0...","[10.648649103590703, 11.64578146419374, 12.676..."
...,...,...,...,...,...,...,...,...,...,...,...,...
47916,1,5,2,2408,1691,11624,23540,25846,"[2023-04-01, 2023-05-01, 2023-06-01, 2023-07-0...","[2, 2, 1, 1, 2, 1, 1]","[2022-01-01, 2022-02-01, 2022-03-01, 2022-07-0...","[0.9448144847801935, 0.892976890340508, 0.8394..."
47917,1,5,2,2408,1691,11624,27374,25846,"[2023-04-01, 2023-05-01, 2023-06-01, 2023-07-0...","[5, 6, 9, 1, 3, 3, 5, 4, 1, 2]","[2022-01-01, 2022-02-01, 2022-03-01, 2022-04-0...","[1.6203798162367373, 1.3702744862746163, 1.111..."
47918,1,5,2,2408,1691,11624,32502,25846,"[2023-04-01, 2023-05-01, 2023-06-01, 2023-07-0...","[4, 1, 1]","[2022-04-01, 2023-02-01, 2023-03-01]","[0.5863184670740451, 0.3066976287548493, 0.017..."
47919,1,5,2,2408,1691,11624,35151,25846,"[2023-04-01, 2023-05-01, 2023-06-01, 2023-07-0...","[1, 1, 1]","[2022-12-01, 2023-02-01, 2023-03-01]","[1.0, 1.0, 1.0, 1.0, 1.0]"


In [44]:
submission = needed_samples_ty[
    ["rps", "podrod", "filial", "client_sap_id", "freight_id", "sender_station_id", "recipient_station_id", "sender_organisation_id", "period_x", "forecast_wagon_count"]
].explode(["period_x", "forecast_wagon_count"])\
.reset_index(drop=True)\
.rename(columns={"period_x": "period"})\

submission

,rps,podrod,filial,client_sap_id,freight_id,sender_station_id,recipient_station_id,sender_organisation_id,period,forecast_wagon_count
0,0,0,0,-1,349,38725,38966,27437,2023-04-01,2.170165
1,0,0,0,-1,349,38725,38966,27437,2023-05-01,0.748055
2,0,0,0,-1,349,38725,38966,27437,2023-06-01,0.0
3,0,0,0,-1,349,38725,38966,27437,2023-07-01,0.0
4,0,0,0,-1,349,38725,38966,27437,2023-08-01,0.0
...,...,...,...,...,...,...,...,...,...,...
239600,1,5,2,2410,1691,32810,11445,25813,2023-04-01,0.949057
239601,1,5,2,2410,1691,32810,11445,25813,2023-05-01,0.781079
239602,1,5,2,2410,1691,32810,11445,25813,2023-06-01,0.607501
239603,1,5,2,2410,1691,32810,11445,25813,2023-07-01,0.439523


In [46]:
NEEDED_COLUMNS = ['period', 'rps', 'podrod', 'filial', 'client_sap_id', 'freight_id',
       'sender_station_id', 'recipient_station_id', 'sender_organisation_id', 'forecast_wagon_count']

submission = submission[NEEDED_COLUMNS]
submission

,period,rps,podrod,filial,client_sap_id,freight_id,sender_station_id,recipient_station_id,sender_organisation_id,forecast_wagon_count
0,2023-04-01,0,0,0,-1,349,38725,38966,27437,2.170165
1,2023-05-01,0,0,0,-1,349,38725,38966,27437,0.748055
2,2023-06-01,0,0,0,-1,349,38725,38966,27437,0.0
3,2023-07-01,0,0,0,-1,349,38725,38966,27437,0.0
4,2023-08-01,0,0,0,-1,349,38725,38966,27437,0.0
...,...,...,...,...,...,...,...,...,...,...
239600,2023-04-01,1,5,2,2410,1691,32810,11445,25813,0.949057
239601,2023-05-01,1,5,2,2410,1691,32810,11445,25813,0.781079
239602,2023-06-01,1,5,2,2410,1691,32810,11445,25813,0.607501
239603,2023-07-01,1,5,2,2410,1691,32810,11445,25813,0.439523


In [51]:
submission["period"] = pd.to_datetime(submission["period"])
submission["forecast_wagon_count"] = submission["forecast_wagon_count"].astype("float")

In [52]:
submission.dtypes

period                    datetime64[ns]
rps                                int64
podrod                             int64
filial                             int64
client_sap_id                      int64
freight_id                         int64
sender_station_id                  int64
recipient_station_id               int64
sender_organisation_id             int64
forecast_wagon_count             float64
dtype: object

In [53]:
submission.to_csv("prophet.csv", index=False, sep=";", decimal=",", encoding="windows-1251")

In [29]:
preds = []

for i in tqdm(range(ty.shape[0])):
    if lens[i] >= 2: 
        df = dfs[i]
        
        m = Prophet()
        m.fit(df)
    
        forecast = m.predict(periods[i])["yhat"]
        
    else:
        forecast = pd.DataFrame({"yhat": [dfs[i]["y"][0]] * 5})

    preds.append(forecast)

  0%|          | 0/47921 [00:00<?, ?it/s]

04:42:01 - cmdstanpy - INFO - Chain [1] start processing
04:42:01 - cmdstanpy - INFO - Chain [1] done processing
04:42:01 - cmdstanpy - INFO - Chain [1] start processing
04:42:01 - cmdstanpy - INFO - Chain [1] done processing
04:42:01 - cmdstanpy - INFO - Chain [1] start processing
04:42:01 - cmdstanpy - INFO - Chain [1] done processing
04:42:01 - cmdstanpy - INFO - Chain [1] start processing
04:42:02 - cmdstanpy - INFO - Chain [1] done processing
04:42:02 - cmdstanpy - INFO - Chain [1] start processing
04:42:02 - cmdstanpy - INFO - Chain [1] done processing
04:42:02 - cmdstanpy - INFO - Chain [1] start processing
04:42:02 - cmdstanpy - INFO - Chain [1] done processing
04:42:02 - cmdstanpy - INFO - Chain [1] start processing
04:42:02 - cmdstanpy - INFO - Chain [1] done processing
04:42:02 - cmdstanpy - INFO - Chain [1] start processing
04:42:02 - cmdstanpy - INFO - Chain [1] done processing
04:42:02 - cmdstanpy - INFO - Chain [1] start processing
04:42:02 - cmdstanpy - INFO - Chain [1]

KeyboardInterrupt: 